## Members

- Ko Choi (kec788)
- Mandeep Burdak (msb4384)
- Nishant Singh (nsk779)
- Pratyush Sharma (ps35484)
- Vaibhav Nagar (vn5339)

In [1]:
import os
import pandas as pd
from tqdm import tqdm
import pytube
import time
from datetime import datetime
import json

import os
import pandas as pd
from tqdm import tqdm
from huggingface_hub import notebook_login
from torch import cuda
from torch import bfloat16
import transformers
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, TextGeneration
from bertopic import BERTopic

from dash import Dash, dcc, html
import plotly.express as px
from base64 import b64encode
import io

import os
import pandas as pd
from tqdm import tqdm
import datetime
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import sent_tokenize, RegexpTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import string

import plotly.express as px
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/drknt_rises/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/drknt_rises/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/drknt_rises/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
current_path = os.getcwd()
current_path = current_path.split('movie-pre-buzz-to-box-office')
path = current_path[0] + 'movie-pre-buzz-to-box-office'
path

## Get Video Description from Youtube.

In [ ]:
videoIds = pd.read_csv(path + "/Data/Trailer Ids/youtube trailer Ids.csv")

def videoDetails(videoId, apiKey):
    import googleapiclient.discovery
    import googleapiclient.errors

    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"
    credentials = apiKey
    
    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey=credentials)

    request = youtube.videos().list(
        part="snippet,contentDetails,statistics", 
        id=videoId
    )
    response = request.execute()

    return response

videoIds = list(videoIds['Video Id'])
key = input("Enter the API Key:")
for i in tqdm(range(len(videoIds))):
    data = []
    row=[]
    row.append(videoIds[i])
    response = videoDetails(videoIds[i], apiKey=key)

    try: 
        row.append(response['etag']) 
    except (IndexError, KeyError) as error: 
        row.append(None)
    
    try:
        row.append(response['items'][0]['snippet']['publishedAt']) 
    except (IndexError, KeyError) as error:
        row.append(None)
    
    try:
        row.append(response['items'][0]['snippet']['title']) 
    except (IndexError, KeyError) as error:
        row.append(None)
    
    try:
        row.append(response['items'][0]['snippet']['description']) 
    except (IndexError, KeyError) as error:
        row.append(None)
    
    try:
        row.append(response['items'][0]['snippet']['channelTitle']) 
    except (IndexError, KeyError) as error:
        row.append(None)
    
    try:
        row.append(response['items'][0]['snippet']['categoryId']) 
    except (IndexError, KeyError) as error:
        row.append(None)

    try:
        row.append(response['items'][0]['snippet']['defaultAudioLanguage'])
    except (IndexError, KeyError) as error:
        row.append(None)
    
    try:
        row.append(response['items'][0]['contentDetails']['duration']) 
    except (IndexError, KeyError) as error:
        row.append(None)
    
    try:
        row.append(response['items'][0]['contentDetails']['definition']) 
    except (IndexError, KeyError) as error:
        row.append(None)
    
    try:
        row.append(response['items'][0]['contentDetails']['caption']) 
    except (IndexError, KeyError) as error:
        row.append(None)
    
    try:
        row.append(response['items'][0]['contentDetails']['licensedContent']) 
    except (IndexError, KeyError) as error:
        row.append(None)
    
    try:
        row.append(response['items'][0]['statistics']['viewCount']) 
    except (IndexError, KeyError) as error:
        row.append(None)
    
    try:
        row.append(response['items'][0]['statistics']['likeCount']) 
    except (IndexError, KeyError) as error:
        row.append(None)
    
    try:
        row.append(response['items'][0]['statistics']['commentCount'])
    except (IndexError, KeyError) as error:
        row.append(None)
    
    data.append(row)

    cols = ['videoId', 'etag', 'publishedAt', 'title', 'description', 'channelTitle', 'categoryId', \
            'defaultAudioLanguage', 'duration', 'definition', 'caption', 'licensedContent', \
                'viewCount', 'likeCount', 'commentCount'] 
    data = pd.DataFrame(data, columns=cols)
    
    if i==0:
        data.to_csv(path + '/Data/Video Description/videoDescription.csv', index=False, header=True)
    else:
        data.to_csv(path + '/Data/Video Description/videoDescription.csv', mode='a', index=False, header=False)

pd.read_csv(path + '/Data/Video Description/videoDescription.csv')

## Download Youtube Videos

In [ ]:
def download_video(url, resolution):
    itag = choose_resolution(resolution)
    video = pytube.YouTube(url)
    stream = video.streams.get_by_itag(itag)
    stream.download()
    return stream.default_filename

def download_videos(urls, resolution):
    for url in urls:
        download_video(url, resolution)

def download_playlist(url, resolution):
    playlist = pytube.Playlist(url)
    download_videos(playlist.video_urls, resolution)

def choose_resolution(resolution):
    if resolution in ["low", "360", "360p"]:
        itag = 18
    elif resolution in ["medium", "720", "720p", "hd"]:
        itag = 22
    elif resolution in ["high", "1080", "1080p", "fullhd", "full_hd", "full hd"]:
        itag = 137
    elif resolution in ["very high", "2160", "2160p", "4K", "4k"]:
        itag = 313
    else:
        itag = 18
    return itag


videoIds = pd.read_csv(path + "/Data/Trailer Ids/youtube trailer Ids.csv")
videoIds = list(videoIds['Video Id'])
for videoId in videoIds:
    download_video(videoId,'low')

## Get Trailer Coments

In [ ]:
def get_comments(videoId, pageToken=False):
    import os
    import googleapiclient.discovery
    import json

    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"
    DEVELOPER_KEY = ""

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)

    if pageToken == False:
        request = youtube.commentThreads().list(
            part="id,snippet,replies",
            maxResults=100,
            order="time",
            textFormat="html",
            videoId=videoId
        )
    else:
        request = youtube.commentThreads().list(
            part="id,snippet,replies",
            maxResults=100,
            order="time",
            videoId=videoId,
            textFormat="html",
            pageToken=pageToken
        )
    
    try:
        response = request.execute()
        code = 200
    except Exception as error:
        code = error.status_code
        response = error.content.decode("utf-8").replace("\n  ","")
        response = json.loads(response)
    return response, code

def remove_tags(html):
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")
 
    for data in soup(['style', 'script']):
        data.decompose()
    
    text = ' '.join(soup.stripped_strings)
    text = text.split("\n")
    text = ' '.join(text) 
    text = text.strip()
    text = text.replace("|", "")
    
    return text
 
videoIds = pd.read_csv(path + "/Data/Trailer Ids/youtube trailer Ids.csv")
videoIds = list(videoIds['Video Id'])[203:]
for videoId in (pbar:= tqdm(videoIds)):
    pbar.set_description(f"Fetching comments from {videoId}")
    response, code = get_comments(videoId=videoId)
    if code != 200:
        data = []
        error_reason = response['error']['errors'][0]['reason']
        data.append([videoId, code, error_reason, None, None, None, None])

        cols = ['videoId', 'code', 'error_reason', 'id', 'replies', 'comment', 'publishedAt']
        data = pd.DataFrame(data, columns=cols)
        data.to_csv(path+'/Data/Comments/Video Comments.csv', index=False, header=False, mode='a', sep='|')
    else:
        start = True
        while start:
            try:
                pageToken = response['nextPageToken']
                for item in response['items']:
                    data = []
                    id = item['id']
                    replies = item['snippet']['totalReplyCount']
                    
                    comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
                    # comment = remove_tags(comment)
                    try:
                        for reply in item['replies']['comments']:
                            reply = reply['snippet']['textDisplay']
                            # reply = remove_tags(reply)
                            comment = comment + ' ' + reply
                    except KeyError:
                        comment = comment
                    
                    comment = remove_tags(comment)
                    
                    publishedAt = item['snippet']['topLevelComment']['snippet']['publishedAt']
                    
                    data.append([videoId, code, None, id, replies, comment, publishedAt])

                    cols = ['videoId', 'code', 'error_reason', 'id', 'replies', 'comment', 'publishedAt']
                    data = pd.DataFrame(data, columns=cols)
                    data.to_csv(path+'/Data/Comments/Video Comments.csv', index=False, header=False, mode='a', sep='|')
            except KeyError:
                start = False
                for item in response['items']:
                    data = []
                    id = item['id']
                    replies = item['snippet']['totalReplyCount']
                    
                    comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
                    # comment = remove_tags(comment)
                    try:
                        for reply in item['replies']['comments']:
                            reply = reply['snippet']['textDisplay']
                            # reply = remove_tags(reply)
                            comment = comment + ' ' + reply
                    except KeyError:
                        comment = comment
                    
                    comment = remove_tags(comment)
                    
                    publishedAt = item['snippet']['topLevelComment']['snippet']['publishedAt']
                    
                    data.append([videoId, code, None, id, replies, comment, publishedAt])

                    cols = ['videoId', 'code', 'error_reason', 'id', 'replies', 'comment', 'publishedAt']
                    data = pd.DataFrame(data, columns=cols)
                    data.to_csv(path+'/Data/Comments/Video Comments.csv', index=False, header=False, mode='a', sep='|')
            
            response, code = get_comments(videoId=videoId, pageToken=pageToken)

pd.read_csv(path+'/Data/Comments/Video Comments.csv', sep='|', lineterminator='\n')

## POST Videos to Azure AI Video Indexer

In [ ]:
def videoURL(fileName, path, filetype, sasToken):
    cont_path = 'https://moviebuzz.blob.core.windows.net/moviebuzztrailers'
    path = cont_path + path + fileName + filetype + sasToken
    return path


def get_accessToken(account_id):
    import urllib.request, json

    try:
        url = "https://api.videoindexer.ai/Auth/trial/Accounts/"+ account_id +"/AccessToken?allowEdit=true"

        hdr ={
        'Ocp-Apim-Subscription-Key': 'c26d838400a54208963e98d0064342fa',
        }

        req = urllib.request.Request(url, headers=hdr)

        req.get_method = lambda: 'GET'
        response = urllib.request.urlopen(req)
        code = response.getcode()
        response = response.read()
        accessToken = json.loads(response)
        return accessToken, code
    except Exception as error:
        code = error.getcode()
        error = error.read()
        error = json.loads(error)
        return error, code


def postVideo(videoURL, fileName, account_id, accessToken, privacy='Private', language='auto'):
    import urllib.request, json
    import time
    
    name = fileName

    base = 'https://api.videoindexer.ai/trial/Accounts/'
    post_url = base + account_id + '/Videos?name=' + name + '&privacy=' + privacy + '&language=' + language + '&videoUrl=' + \
        videoURL + '&fileName=' + fileName + '&accessToken=' + accessToken

    try:
        hdr ={
        'Cache-Control': 'no-cache',
        }

        req = urllib.request.Request(post_url, headers=hdr)

        req.get_method = lambda: 'POST'
        response = urllib.request.urlopen(req)
        code = response.getcode()
        response = response.read()
        response = json.loads(response)
        return response, code, post_url, accessToken
    except Exception as error:
        code = error.getcode()
        if code == 401:
            time.sleep(3)
            accessToken, _ = get_accessToken(account_id=account_id)

            post_url = base + account_id + '/Videos?name=' + name + '&privacy=' + privacy + '&language=' + language + '&videoUrl=' + \
                        videoURL + '&fileName=' + fileName + '&accessToken=' + accessToken
            
            try:
                hdr = {
                    'Cache-Control': 'no-cache',
                }

                req = urllib.request.Request(post_url, headers=hdr)

                req.get_method = lambda: 'POST'
                response = urllib.request.urlopen(req)
                code = response.getcode()
                response = response.read()
                response = json.loads(response)
                return response, code, post_url, accessToken
            except Exception as error:
                code = error.getcode()
                error = error.read()
                error = json.loads(error)
                return error, code, post_url, accessToken
        else:
            error = error.read()
            error = json.loads(error)
            return error, code, post_url, accessToken

videoIds = pd.read_csv(path + '/Data/Trailer Ids/youtube trailer Ids.csv')
videoIds = list(videoIds['Video Id'])
sasToken = '?sp=r&st=2023-10-02T18:44:01Z&se=2023-10-06T02:44:01Z&skoid=59bac173-f573-4e6d-bccd-0c73ef423d4d&sktid=eb8a7f2b-13aa-40dc-845e-67f52adc533d&skt=2023-10-02T18:44:01Z&ske=2023-10-06T02:44:01Z&sks=b&skv=2022-11-02&spr=https&sv=2022-11-02&sr=c&sig=UxRExr3BScdwgOG3kp%2ByUSCKtkB69yI80IzvBPZX5Po%3D'
account_id = "7a8b1d65-2c64-4a2d-a275-eb72026881c9"
access_token, sas_code = get_accessToken(account_id=account_id)
for i in (pbar := tqdm(range(len(videoIds)))):
    data = []
    row = []
    
    videoId = videoIds[i]
    pbar.set_description(f"Uploading {videoId}")
    row.append(videoId)
    
    video_url = videoURL(fileName=videoId, path='/trailers/', filetype='.mp4', sasToken=sasToken)
    row.append(video_url)

    # Change videURL to use real one
    response, code, post_url, access_token = postVideo(videoURL=video_url, fileName=videoId, account_id=account_id, accessToken=access_token)
    row.append(code)
    row.append(post_url)
    row.append(response)
    try:
        row.append(response['id'])
    except:
        row.append(None)
    
    try:
        row.append(response['created'])
    except:
        row.append(None)
    
    current_datetime = datetime.utcnow()
    row.append(current_datetime)

    data.append(row)
    cols = ['VideoId', 'VideoUrl', 'ResponseCode', 'PostUrl', 'Response', 'Id', 'UploadTime', 'CreatedTime']
    data = pd.DataFrame(data, columns=cols)
    if i==0:
        data.to_csv(path+'/Data/Upload Data/uploadData.csv', index=False, sep='|', header=False, mode='a')
    else:
        data.to_csv(path+'/Data/Upload Data/uploadData.csv', index=False, sep='|', mode='a', header=False)
    time.sleep(5)

left_uploads = ['0']
while len(left_uploads) != 0:
    uploads = pd.read_csv(path+'/Data/Upload Data/uploadData.csv', sep='|').sort_values("CreatedTime", ascending=False)
    uploads = uploads.loc[uploads['ResponseCode'] == 200]
    uploads = uploads.drop_duplicates(['VideoId'], keep='first')
    uploads = set(uploads['VideoId'])
    left_uploads = set(videoIds) - uploads
    left_uploads = list(left_uploads)
    sasToken = ''
    account_id = ""
    access_token, sas_code = get_accessToken(account_id=account_id)
    for i in (pbar := tqdm(range(len(left_uploads)))):
        data = []
        row = []
        
        videoId = left_uploads[i]
        pbar.set_description(f"Uploading {videoId}")
        row.append(videoId)
        
        video_url = videoURL(fileName=videoId, path='/trailers/', filetype='.mp4', sasToken=sasToken)
        row.append(video_url)

        # Change videURL to use real one
        response, code, post_url, access_token = postVideo(videoURL=video_url, fileName=videoId, account_id=account_id, accessToken=access_token)
        row.append(code)
        row.append(post_url)
        row.append(response)
        try:
            row.append(response['id'])
        except:
            row.append(None)
        
        try:
            row.append(response['created'])
        except:
            row.append(None)
        
        current_datetime = datetime.utcnow()
        row.append(current_datetime)

        data.append(row)
        cols = ['VideoId', 'VideoUrl', 'ResponseCode', 'PostUrl', 'Response', 'Id', 'UploadTime', 'CreatedTime']
        data = pd.DataFrame(data, columns=cols)
        data.to_csv(path+'/Data/Upload Data/uploadData.csv', index=False, sep='|', header=False, mode='a')
        
        if code != 200:
            time.sleep(50)

    uploads = pd.read_csv(path+'/Data/Upload Data/uploadData.csv', sep='|').sort_values("CreatedTime", ascending=False)
    uploads = uploads.loc[uploads['ResponseCode'] == 200]
    uploads = uploads.drop_duplicates(['VideoId'], keep='first')
    uploads = set(uploads['VideoId'])
    left_uploads = set(videoIds) - uploads
    left_uploads = list(left_uploads)

uploads = pd.read_csv(path+'/Data/Upload Data/uploadData.csv', sep='|').sort_values("CreatedTime", ascending=False)
uploads = uploads.loc[uploads['ResponseCode'] == 200]
uploads = uploads.drop_duplicates(['VideoId'], keep='first')
uploads

## GET Indexed Video Details

In [ ]:
def get_accessToken(account_id):
    import urllib.request, json

    try:
        url = "https://api.videoindexer.ai/Auth/trial/Accounts/"+ account_id +"/AccessToken?allowEdit=true"

        hdr ={
        'Ocp-Apim-Subscription-Key': 'c26d838400a54208963e98d0064342fa',
        }

        req = urllib.request.Request(url, headers=hdr)

        req.get_method = lambda: 'GET'
        response = urllib.request.urlopen(req)
        code = response.getcode()
        response = response.read()
        accessToken = json.loads(response)
        return accessToken, code
    except Exception as error:
        code = error.getcode()
        error = error.read()
        error = json.loads(error)
        return error, code
    

def VideoIndex(account_id, videoid, accessToken, language='en-US', retranslate='false', includeStreamingUrls='true', includeSummarizedInsights='true'):
    import urllib.request, json

    try:
        base = "https://api.videoindexer.ai/trial/Accounts/"
        get_url = base + account_id + "/Videos/" + videoid + "/Index?language=" + language + "&reTranslate=" + retranslate + "&includeStreamingUrls=" + includeStreamingUrls + "&includeSummarizedInsights=" + includeSummarizedInsights + "&accessToken=" + accessToken

        hdr ={
        'Cache-Control': 'no-cache',
        'Ocp-Apim-Subscription-Key': 'c26d838400a54208963e98d0064342fa',
        }

        req = urllib.request.Request(get_url, headers=hdr)

        req.get_method = lambda: 'GET'
        response = urllib.request.urlopen(req)
        code = response.getcode()
        response = response.read()
        response = json.loads(response)
        return response, code, get_url, accessToken
    except Exception as error:
        code = error.getcode()
        error = error.read()
        error = json.loads(error)
        return error, code, get_url, accessToken

In [ ]:
uploads = pd.read_csv(path+'/Data/Upload Data/uploadData.csv', sep='|').sort_values("CreatedTime", ascending=False)
uploads = uploads.loc[uploads['ResponseCode'] == 200]
uploads = uploads.drop_duplicates(['VideoId'], keep='first')
videoIds = {}
for videoId, fileId in zip(uploads['VideoId'], uploads['Id']):
    videoIds[fileId] = videoId
fileIds = list(uploads['Id'])

recieved = pd.read_csv(path+'/Data/Video Indexes/Raw Data.csv', sep='|').sort_values("CreatedTime", ascending=False)
recieved = recieved.loc[recieved['ResponseCode'] == 200]
recieved = recieved.drop_duplicates(['VideoId'], keep='first')
recieved = set(recieved['FileId'])
left = set(fileIds) - recieved
left = list(left)
len(left)

while len(left) != 0:
    account_id = "7a8b1d65-2c64-4a2d-a275-eb72026881c9"
    access_token, sas_code = get_accessToken(account_id=account_id)
    for i in (pbar := tqdm(range(len(left)))):
        data = []
        row = []
        row.append(videoIds[left[i]])

        pbar.set_description(f"Recieving {left[i]}")
        file_id = left[i]
        row.append(file_id)

        response, code, get_url, access_token = VideoIndex(account_id=account_id, videoid=file_id, accessToken=access_token)
        row.append(code)
        row.append(get_url)
        
        if code == 200:
            obj = json.dumps(response, indent=3)
            with open(f"{path}/Data/Video Indexes/JSON/{videoIds[left[i]]}.json", "w") as fp:
                fp.write(obj) 
            row.append('saved')
        else:
            row.append('failed')

        current_datetime = datetime.utcnow()
        row.append(current_datetime)

        data.append(row)
        cols = ['VideoId', 'FileId', 'ResponseCode', 'GetUrl', 'Response', 'CreatedTime']
        data = pd.DataFrame(data, columns=cols)
        data.to_csv(path+'/Data/Video Indexes/Raw Data.csv', index=False, sep='|', header=False, mode='a')
        
        if code != 200:
            time.sleep(61)
        
    recieved = pd.read_csv(path+'/Data/Video Indexes/Raw Data.csv', sep='|').sort_values("CreatedTime", ascending=False)
    recieved = recieved.loc[recieved['ResponseCode'] == 200]
    recieved = recieved.drop_duplicates(['VideoId'], keep='first')
    recieved = set(recieved['FileId'])
    left = set(fileIds) - recieved
    left = list(left)

video_indices = pd.read_csv(path+'/Data/Video Indexes/Raw Data.csv', sep='|').sort_values("CreatedTime", ascending=False)
video_indices = video_indices.loc[video_indices['ResponseCode'] == 200]
video_indices = video_indices.drop_duplicates(['VideoId'], keep='first')
video_indices

## Generate Video Description

In [ ]:
def process_prompt(prompt):
        import openai
        from openai.error import ServiceUnavailableError, APIError, Timeout, AuthenticationError

        openai_key = ''
        openai.api_key = openai_key

        try:
            # @backoff.on_exception(backoff.expo, (ServiceUnavailableError, APIError))
            def chat_completion(prompt):
                response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                        {"role": "system", "content": prompt},
                    ]
                )

                response = response['choices'][0]['message']['content']
                return response
            
            response = chat_completion(prompt)
            return response, 200
        except (ServiceUnavailableError, APIError, Timeout) as error:
            return 'error', 400

videoIds = pd.read_csv(path + "/Data/Trailer Ids/youtube trailer Ids.csv")
videodescriptions = pd.read_csv(path + "/Data/Video Description/videoDescription.csv")

videodescriptions_dict = {}
for videoId, description in zip(videodescriptions['videoId'], videodescriptions['description']):
    videodescriptions_dict[videoId] = description

videodescriptions_dict

videoIds = list(videoIds['Video Id'])
# videoIds = ['iBgubizv64g','XKeNWUsC_Nw','BoNynvcG7Ok','b6qSGKN42Gc','tSkv_sEyPSc','gSMxBLlA8qY','Na6gA1RehsU','09zYzvKkCOA']
for videoId in (pbar:= tqdm(videoIds)):
    pbar.set_description(f"Generating video description for {videoId}")
    final_data = []
    with open((path + f"/Data/Video Indexes/JSON/{videoId}.json"), 'r') as fp:
        data = json.load(fp)

        keywords = []
        for keyword in data['summarizedInsights']['keywords']:
            keywords.append(keyword['name'])

        sentiments = []
        for sentiment in data['summarizedInsights']['sentiments']:
            if sentiment['sentimentKey'] != 'Neutral':
                sentiments.append([sentiment['sentimentKey'], sentiment['seenDurationRatio']])
        
        emotions = []
        for emotion in data['summarizedInsights']['emotions']:
            emotions.append([emotion['type'], emotion['seenDurationRatio']])
        
        audioeffects = []
        for audioeffect in data['summarizedInsights']['audioEffects']:
            audioeffects.append([audioeffect['audioEffectKey'], audioeffect['seenDurationRatio']])
        
        labels = []
        for label in data['summarizedInsights']['labels']:
            labels.append([label['name'], len(label['appearances'])])
        
        framepatters = []
        for pattern in data['summarizedInsights']['framePatterns']:
            sum = 0
            for appearence in pattern['appearances']:
                sum = sum + float(appearence['endSeconds']) - float(appearence['startSeconds'])
            sum = round(sum, 2)
            framepatters.append([pattern['name'], len(pattern['appearances']), sum])
        
        topics = []
        for topic in data['summarizedInsights']['topics']:
            topics.append(topic['name'])


    i = 0
    sentiment_passer = sentiments.copy()
    for sentiment in sentiments:
        sentiments[i] = f'"{sentiment[0]}" seen for {round(sentiment[1]*100, 2)}%'
        i += 1

    i = 0
    for emotion in emotions:
        emotions[i] = f'"{emotion[0]}" seen for {round(emotion[1]*100, 2)}%'
        i += 1

    i = 0
    for audioeffect in audioeffects:
        audioeffects[i] = f'"{audioeffect[0]}" seen for {round(audioeffect[1]*100, 2)}%'
        i += 1

    i = 0
    for label in labels:
        labels[i] = f'"{label[0]}" seen {label[1]} times'
        i += 1

    i = 0
    for framepatter in framepatters:
        framepatters[i] = f'"{framepatter[0]}" frame seen {framepatter[1]} times for a total of {framepatter[2]} seconds'
        i += 1
    
    prompt = f'''Create a humanized description of a movie trailer which has the followind identified attributes:
    1. Keywords indentified in the video: {', '.join(keywords)}
    2. Sentiments observed in the video: {', '.join(sentiments)}
    3. Emotions seen in the video: {', '.join(emotions)}
    4. Audio Effects present in the video: {', '.join(audioeffects)}
    5. Attributes/Objects sighted in the video: {', '.join(labels)}
    6. Frame Patterns: {', '.join(framepatters)}
    7. Topics inferred: {', '.join(topics)}
    8. Trailer description: "{videodescriptions_dict[videoId]}"

Make sure there are not links in the result.'''

    prompt

    description, code = process_prompt(prompt=prompt)
    # description = description.strip()
    
    current_datetime = datetime.utcnow()
    final_data.append([videoId, code, prompt.replace('\n', ''), description, ','.join(sentiment_passer), current_datetime])
    
    cols = ['VideoId', 'Code', 'Prompt', 'OpenAIGeneratedDescription', 'CreatedTime']
    final_data = pd.DataFrame(final_data, columns=cols)
    final_data.to_csv(path+'/Data/Processed/ProcessedVideoDescription.csv', index=False, sep='|', header=False, mode='a', lineterminator='%')

generated_description = pd.read_csv(path+'/Data/Processed/ProcessedVideoDescription.csv', sep='|', lineterminator='%')
generated_description

## Topic Modelling w BERT & Llama 2

In [ ]:
notebook_login()

model_id = 'meta-llama/Llama-2-13b-chat-hf'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

print(device)

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
)
model.eval()

generator = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    temperature=0.1,
    max_new_tokens=500,
    repetition_penalty=1.1
)

prompt = "How to make social media posts engaging?"
res = generator(prompt)
print(res[0]["generated_text"])

system_prompt = """
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant for labeling topics.
<</SYS>>
"""

example_prompt = """
I have a topic that contains the following documents:
- Traditional diets in most cultures were primarily plant-based with a little meat on top, but with the rise of industrial style meat production and factory farming, meat has become a staple food.
- Meat, but especially beef, is the word food in terms of emissions.
- Eating meat doesn't make you a bad person, not eating meat doesn't make you a good one.

The topic is described by the following keywords: 'meat, beef, eat, eating, emissions, steak, food, health, processed, chicken'.

Based on the information about the topic above, please create a short label of this topic. Make sure you to only return the label and nothing more.

[/INST] Environmental impacts of eating meat
"""

main_prompt = """
[INST]
I have a topic that contains the following documents:
[DOCUMENTS]

The topic is described by the following keywords: '[KEYWORDS]'.

Based on the information about the topic above, please create a short label of this topic. Make sure you to only return the label and nothing more.
[/INST]
"""

prompt = system_prompt + example_prompt + main_prompt
print(prompt)

generated_description = pd.read_csv(path+'/Data/Processed/ProcessedVideoDescription.csv',  sep='|', lineterminator='%')
video_description = pd.read_csv(path+'/Data/Video Description/videoDescription.csv')

video_description = video_description[['videoId', 'title']]
generated_description = generated_description.merge(video_description, left_on='VideoId', right_on='videoId', how='left')

generated_description['CreatedTime'] = pd.to_datetime(generated_description['CreatedTime'])
generated_description = generated_description.sort_values('CreatedTime', ascending=False).drop_duplicates('VideoId', keep='first').reset_index(drop=True)

descriptions = generated_description['OpenAIGeneratedDescription']
title = generated_description['title']

print(descriptions[201])

embedding_model = SentenceTransformer("BAAI/bge-large-en")
embeddings = embedding_model.encode(descriptions, show_progress_bar=True)

umap_model = UMAP(n_neighbors=4, n_components=9, min_dist=0.0, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=4, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

reduced_embeddings = UMAP(n_neighbors=4, n_components=2, min_dist=0.0, metric='cosine', random_state=42).fit_transform(embeddings)

keybert = KeyBERTInspired()
mmr = MaximalMarginalRelevance(diversity=0.3)
llama2 = TextGeneration(generator, prompt=prompt)

representation_model = {
    "KeyBERT": keybert,
    "Llama2": llama2,
    "MMR": mmr,
}

from bertopic import BERTopic

topic_model = BERTopic(
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  # nr_topics='auto',
  representation_model=representation_model,

  top_n_words=10,
  verbose=True
)

topics, probs = topic_model.fit_transform(descriptions, embeddings)

iden_topics = topic_model.get_topic_info()
iden_topics.to_csv('iden_topics.csv', index=False)
iden_topics

videoId = list(generated_description['videoId'])
red_embeddings = list(reduced_embeddings)
probabilities = list(probs)

data = []
for i, topic in enumerate(topics):
  data.append([videoId[i], topic, probabilities[i], red_embeddings[i][0], red_embeddings[i][1]])

data = pd.DataFrame(data, columns=['VideoId', 'Topic', 'Probaibility', 'ReducedEmbeddings_x', 'ReducedEmbeddings_y'])
data.to_csv('topic_classification.csv', index=False)
data

topic_model.get_topic(1, full=True)["KeyBERT"]

llama2_labels = [label[0][0].split("\n")[0] for label in topic_model.get_topics(full=True)["Llama2"].values()]
topic_model.set_topic_labels(llama2_labels)

plot = topic_model.visualize_documents(title, reduced_embeddings=reduced_embeddings, hide_annotations=True, hide_document_hover=False, custom_labels=True)
import plotly.offline as pyo
pyo.plot(plot, filename='plotly_plot_topics.html')
plot



## Final Analysis

In [9]:
current_path = os.getcwd()
current_path = current_path.split('movie-pre-buzz-to-box-office')
path = current_path[0] + 'movie-pre-buzz-to-box-office'
path

video_description = pd.read_csv(path+'/Data/Video Description/videoDescription.csv').drop_duplicates()
video_description['publishedAt'] = pd.to_datetime(video_description['publishedAt'])

iden_tpoics = pd.read_csv(path+'/Data/Topic Modelling/iden_topics.csv')
topic_classification = pd.read_csv(path+'/Data/Topic Modelling/topic_classification.csv')

comments = pd.read_csv(path+'/Data/Comments/Video Comments.csv', sep='|', lineterminator='\n').drop_duplicates()
comments['publishedAt'] = pd.to_datetime(comments['publishedAt'])
comments = comments.loc[comments['code'] == 200]
comments = comments.rename(columns={'publishedAt': 'postedAt'})
comments

video_description = video_description[['videoId', 'publishedAt']]
comments = comments.merge(video_description, left_on='videoId', right_on='videoId', how='left')
comments['cutoffdate'] = comments['publishedAt'] + datetime.timedelta(days=21)
comments = comments.loc[comments['postedAt'] < comments['cutoffdate']]
comments

def replace_punct(text):
    text = str(text)
    for i in list(string.punctuation):
        text = text.replace(i, ' ')
    text = text.lower()
    return text

comments['comment'] = comments['comment'].apply(lambda row: str(row).lower())
stop_words = stopwords.words('english')
comments['comment'] = comments['comment'].apply(lambda row: ' '.join([word for word in row.split() if word not in stop_words]))
comments['comment'] = comments['comment'].apply(lambda row: replace_punct(row))
comments

comments['comment'] = comments['comment'].apply(lambda row: sent_tokenize(row))
comments['comment'] = comments['comment'].apply(lambda row: ' '.join(row))
comments

sent = SentimentIntensityAnalyzer()
comments['score'] = comments['comment'].apply(lambda row: sent.polarity_scores(row))

comments['negative'] = comments['score'].apply(lambda row: row['neg'])
comments['neutral'] = comments['score'].apply(lambda row: row['neu'])
comments['positive'] = comments['score'].apply(lambda row: row['pos'])
comments['compound'] = comments['score'].apply(lambda row: row['compound'])

comments

def polarity_score(compound):
    if compound > 0.05:
        return "positive"
    elif compound < -0.5:
        return "negative"
    elif compound >= -0.05 and compound < 0.05:
        return "neutral"

comments['sentiment'] = comments['compound'].apply(lambda val: polarity_score(val))
comments

comments_ = comments.groupby('videoId').agg(
    positive_sent = pd.NamedAgg('sentiment', aggfunc=lambda x: (x == 'positive').sum()),
    negative_sent = pd.NamedAgg('sentiment', aggfunc=lambda x: (x == 'negative').sum()),
    neutral_sent = pd.NamedAgg('sentiment', aggfunc=lambda x: (x == 'neutral').sum()),
    total_comments = pd.NamedAgg('videoId', aggfunc='count')
).reset_index(drop=False)

comments_['pos-neg'] = np.log(comments_['positive_sent']) / np.log(comments_['negative_sent'])
comments_

video_description = pd.read_csv(path+'/Data/Video Description/videoDescription.csv').drop_duplicates()
video_description['publishedAt'] = pd.to_datetime(video_description['publishedAt'])

topic_classification = topic_classification.merge(iden_tpoics[['Topic', 'CustomName']], left_on='Topic', right_on='Topic', how='left')
topic_classification = topic_classification.merge(video_description[['videoId', 'title']], left_on='VideoId', right_on='videoId', how='left')
topic_classification_passer = topic_classification.copy()
topic_classification

comments_ = comments_.loc[comments_['pos-neg'] != np.inf]
final = topic_classification.merge(comments_, left_on='VideoId', right_on='videoId', how='left')
final = final.loc[final['pos-neg'] > 0]
final['ReducedEmbeddings'] = [[i, j] for i, j in zip(final['ReducedEmbeddings_x'], final['ReducedEmbeddings_y'])]

generated_description = pd.read_csv(path+'/Data/Processed/ProcessedVideoDescription.csv',  sep='|', lineterminator='%')
generated_description = generated_description.sort_values('CreatedTime', ascending=False).drop_duplicates('VideoId', keep='first').reset_index(drop=True)
final = final.merge(generated_description[['VideoId', 'OpenAIGeneratedDescription', ]], left_on='VideoId', right_on='VideoId', how='left')
final

a = final.groupby('CustomName').agg(
    avg_pos_neg = pd.NamedAgg('pos-neg', aggfunc='mean'),
    count = pd.NamedAgg('total_comments', aggfunc='sum')
).sort_values('avg_pos_neg', ascending=False)
a = a.reset_index(drop=False)
display(a.sort_values('count', ascending=False))

dis = final.loc[final['CustomName'] == 'Emotional Journeys Through Life and History']
dis = dis.sort_values('total_comments', ascending=False)[:1]
display(dis)

display(px.scatter(final, x='ReducedEmbeddings_x', y='ReducedEmbeddings_y', width=1200, height=750, hover_data=['title', 'pos-neg'], color='CustomName'))

final.to_csv('final_reg_data.csv', index=False)

commets__ = comments[['videoId', 'comment', 'sentiment']].merge(topic_classification_passer.drop(['videoId'], axis=1), left_on='videoId', right_on='VideoId', how='left')

commets__ = commets__.groupby('CustomName').agg(
    positive_sent = pd.NamedAgg('sentiment', aggfunc=lambda x: (x == 'positive').sum()),
    negative_sent = pd.NamedAgg('sentiment', aggfunc=lambda x: (x == 'negative').sum()),
    neutral_sent = pd.NamedAgg('sentiment', aggfunc=lambda x: (x == 'neutral').sum()),
    total_comments = pd.NamedAgg('videoId', aggfunc='count')
).reset_index(drop=False)

commets__['pos-neg'] = np.log(comments_['positive_sent']) / np.log(comments_['negative_sent'])
display(commets__)

commets__['pos_neg_mod'] = [i if i>0 else 0 for i in commets__['pos-neg']]
display(commets__)

final2 = final.merge(commets__[['CustomName', 'pos_neg_mod']], left_on=['CustomName'], right_on=['CustomName'], how='left')
display(final2)

display(px.scatter(final2, x='ReducedEmbeddings_x', y='ReducedEmbeddings_y', width=1200, height=750, hover_data=['title', 'CustomName'], color='pos_neg_mod'))

display(final.loc[final['title'] == 'Murder on the Orient Express | Official Trailer [HD] | 20th Century FOX'])

display(comments.loc[comments['videoId'] == 'Mq4m3yAoW8E'])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning:

divide by zero encountered in log



,CustomName,avg_pos_neg,count
9,Movie Trailers,1.973527,63574.0
10,Emotional Journeys Through Life and History,1.957723,16191.0
14,Movies and Truth,1.849617,11606.0
24,Empowering Stories of Resilience,1.631887,11485.0
5,Legendary Athletes in Film,2.421624,9182.0
28,Horror and Ghost Stories,1.207589,8472.0
19,Mafia and Organized Crime,1.715849,7342.0
18,Inspirational True Stories,1.750015,6927.0
15,Coming-of-age films,1.813683,6105.0
7,Love and Loss Emotional Journeys,2.037886,5725.0


,VideoId,Topic,Probaibility,ReducedEmbeddings_x,ReducedEmbeddings_y,CustomName,videoId_x,title,videoId_y,positive_sent,negative_sent,neutral_sent,total_comments,pos-neg,ReducedEmbeddings,OpenAIGeneratedDescription
243,uYPbbksJxIg,18,0.59152,3.114096,4.091232,Emotional Journeys Through Life and History,uYPbbksJxIg,Oppenheimer | New Trailer,uYPbbksJxIg,4537.0,2017.0,5287.0,13348.0,1.106534,"[3.1140964, 4.091232]",Get ready for a thrilling journey through hist...


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning:

divide by zero encountered in log



,CustomName,positive_sent,negative_sent,neutral_sent,total_comments,pos-neg
0,Adoption and Family Dynamics,79,8,52,146,2.929947
1,Biographies and Historical Dramas,1039,165,697,2094,-0.000000
2,Coming-of-age films,2836,420,2313,6157,2.020559
3,Emotional Journeys,230,70,156,495,1.093524
4,Emotional Journeys Through Life and History,5754,2244,6472,16231,1.470747
5,Empowering Stories of Resilience,5640,887,3894,11509,1.278464
6,Heartwarming Friendship Stories,414,71,461,1062,-0.000000
7,Heartwarming Movie Releases,1344,194,1132,3052,-0.000000
8,Horror and Ghost Stories,3224,799,3495,8529,-0.000000
9,Innovators and Their Dreams,205,36,155,426,-0.000000


,CustomName,positive_sent,negative_sent,neutral_sent,total_comments,pos-neg,pos_neg_mod
0,Adoption and Family Dynamics,79,8,52,146,2.929947,2.929947
1,Biographies and Historical Dramas,1039,165,697,2094,-0.000000,0.000000
2,Coming-of-age films,2836,420,2313,6157,2.020559,2.020559
3,Emotional Journeys,230,70,156,495,1.093524,1.093524
4,Emotional Journeys Through Life and History,5754,2244,6472,16231,1.470747,1.470747
5,Empowering Stories of Resilience,5640,887,3894,11509,1.278464,1.278464
6,Heartwarming Friendship Stories,414,71,461,1062,-0.000000,0.000000
7,Heartwarming Movie Releases,1344,194,1132,3052,-0.000000,0.000000
8,Horror and Ghost Stories,3224,799,3495,8529,-0.000000,0.000000
9,Innovators and Their Dreams,205,36,155,426,-0.000000,0.000000


,VideoId,Topic,Probaibility,ReducedEmbeddings_x,ReducedEmbeddings_y,CustomName,videoId_x,title,videoId_y,positive_sent,negative_sent,neutral_sent,total_comments,pos-neg,ReducedEmbeddings,OpenAIGeneratedDescription,pos_neg_mod
0,Na6gA1RehsU,0,1.000000,7.812714,3.023268,Movie Trailers,Na6gA1RehsU,TÁR - Official Trailer [HD] - In Select Theate...,Na6gA1RehsU,102.0,13.0,48.0,179.0,1.803144,"[7.812714, 3.0232685]",TÁR is a captivating and visually stunning fil...,0.000000
1,gSMxBLlA8qY,17,0.673917,5.310845,2.776821,True Crime Dramas,gSMxBLlA8qY,RICHARD JEWELL - Official Trailer [HD],gSMxBLlA8qY,471.0,193.0,273.0,1086.0,1.169527,"[5.310845, 2.7768207]","In the upcoming film ""Richard Jewell,"" directe...",1.310565
2,nrXlY6gzTTM,20,0.312142,1.904737,2.963583,True stories of historical figures,nrXlY6gzTTM,The Post | Official Trailer [HD] | 20th Centur...,nrXlY6gzTTM,371.0,55.0,254.0,764.0,1.476344,"[1.9047374, 2.963583]","In this thrilling drama, legendary filmmaker S...",2.000000
3,XFYWazblaUA,18,0.760441,2.787400,3.801875,Emotional Journeys Through Life and History,XFYWazblaUA,THE SHAPE OF WATER | Official Trailer | FOX Se...,XFYWazblaUA,1181.0,224.0,1168.0,2826.0,1.307202,"[2.7874, 3.8018749]","Introducing ""The Shape of Water,"" a captivatin...",1.470747
4,-Denciie5oA,3,0.990479,2.245054,2.972476,Movies and their themes,-Denciie5oA,12 STRONG - Official Trailer,-Denciie5oA,430.0,256.0,441.0,1291.0,1.093524,"[2.245054, 2.9724758]",Get ready for an intense and action-packed rid...,1.358033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,AHmCH7iB_IM,0,1.000000,6.514732,2.210091,Movie Trailers,AHmCH7iB_IM,CREED III | Official Trailer,AHmCH7iB_IM,2989.0,335.0,3017.0,7011.0,1.376421,"[6.514732, 2.2100906]","In the official trailer for ""Creed III,"" direc...",0.000000
241,bRDuc3IdOn8,15,1.000000,6.107909,3.331384,Mystery and Suspense Movies,bRDuc3IdOn8,The Painted Bird - Official Trailer I HD I IFC...,bRDuc3IdOn8,17.0,3.0,4.0,27.0,2.578902,"[6.1079087, 3.331384]",Experience the gripping and visually stunning ...,1.366309
242,eYfKlNBLLeQ,22,0.671339,4.078673,2.577462,Emotional Journeys,eYfKlNBLLeQ,Little Joe - Official Trailer,eYfKlNBLLeQ,12.0,3.0,9.0,26.0,2.261860,"[4.078673, 2.577462]","In the movie trailer for ""Little Joe,"" we are ...",1.093524
243,uYPbbksJxIg,18,0.591520,3.114096,4.091232,Emotional Journeys Through Life and History,uYPbbksJxIg,Oppenheimer | New Trailer,uYPbbksJxIg,4537.0,2017.0,5287.0,13348.0,1.106534,"[3.1140964, 4.091232]",Get ready for a thrilling journey through hist...,1.470747


,VideoId,Topic,Probaibility,ReducedEmbeddings_x,ReducedEmbeddings_y,CustomName,videoId_x,title,videoId_y,positive_sent,negative_sent,neutral_sent,total_comments,pos-neg,ReducedEmbeddings,OpenAIGeneratedDescription
33,Mq4m3yAoW8E,14,1.0,4.78656,2.202778,Movies and Truth,Mq4m3yAoW8E,Murder on the Orient Express | Official Traile...,Mq4m3yAoW8E,1563.0,484.0,1303.0,3772.0,1.189625,"[4.78656, 2.2027779]","Experience the breathtaking journey of ""Murder..."


,videoId,code,error_reason,id,replies,comment,postedAt,publishedAt,cutoffdate,score,negative,neutral,positive,compound,sentiment
304267,Mq4m3yAoW8E,200,NaN,UghNUCVn1vGe_XgCoAEC,0.0,johnny depp penelope cruz awesome film,2017-06-22 12:57:51+00:00,2017-06-01 13:00:34+00:00,2017-06-22 13:00:34+00:00,"{'neg': 0.0, 'neu': 0.549, 'pos': 0.451, 'comp...",0.000,0.549,0.451,0.6249,positive
304268,Mq4m3yAoW8E,200,NaN,UghfmllpF5mNnngCoAEC,0.0,waiting say i ve read book movie,2017-06-22 10:41:36+00:00,2017-06-01 13:00:34+00:00,2017-06-22 13:00:34+00:00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral
304269,Mq4m3yAoW8E,200,NaN,Ugh1iotOB3QueXgCoAEC,0.0,read book summer and climax struck surprise ...,2017-06-22 09:04:46+00:00,2017-06-01 13:00:34+00:00,2017-06-22 13:00:34+00:00,"{'neg': 0.134, 'neu': 0.47, 'pos': 0.396, 'com...",0.134,0.470,0.396,0.5994,positive
304270,Mq4m3yAoW8E,200,NaN,UgiwS87OE3-YO3gCoAEC,0.0,colonel arbuthnot,2017-06-22 03:18:17+00:00,2017-06-01 13:00:34+00:00,2017-06-22 13:00:34+00:00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral
304271,Mq4m3yAoW8E,200,NaN,Ugj3v4gupZAR_XgCoAEC,0.0,good song go trailer instead believer human ...,2017-06-22 02:15:13+00:00,2017-06-01 13:00:34+00:00,2017-06-22 13:00:34+00:00,"{'neg': 0.124, 'neu': 0.725, 'pos': 0.15, 'com...",0.124,0.725,0.150,0.1280,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308034,Mq4m3yAoW8E,200,NaN,UggcI6IH82J1EngCoAEC,0.0,sub nd sub,2017-06-01 13:05:39+00:00,2017-06-01 13:00:34+00:00,2017-06-22 13:00:34+00:00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral
308035,Mq4m3yAoW8E,200,NaN,UgjFt0C9KxWotHgCoAEC,0.0,cool,2017-06-01 13:05:12+00:00,2017-06-01 13:00:34+00:00,2017-06-22 13:00:34+00:00,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...",0.000,0.000,1.000,0.3182,positive
308036,Mq4m3yAoW8E,200,NaN,UgjzA_xQKnvyS3gCoAEC,0.0,stylelized perfect,2017-06-01 13:03:21+00:00,2017-06-01 13:00:34+00:00,2017-06-22 13:00:34+00:00,"{'neg': 0.0, 'neu': 0.213, 'pos': 0.787, 'comp...",0.000,0.213,0.787,0.5719,positive
308037,Mq4m3yAoW8E,200,NaN,UgjFC_IxHt3SrngCoAEC,0.0,100 club,2017-06-01 13:03:07+00:00,2017-06-01 13:00:34+00:00,2017-06-22 13:00:34+00:00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral
